In [1]:
#Functions to implement analytic prices of ESO as given in 'Analytic Pricing of Employee Stock Options'

#CaseB refers to the model with a single departure process, no barrier and no vesting
#CaseC refers to model with one departure process, and a barrier
#CaseD refers to model with two departure processes, one before and one after vesting, as well as barrier

#NOTE: CaseD requires the distribution function of a bivariate normal RV, which must be approximated so 
#values of CaseD may vary from other sources depending on the approximation method used

#NOTE: CaseD requires T_0 to be strictly positive

#Input parameters here:

s = 100. #Current stock price
K = 100. #Strike price
L = 150. #Level of barrier
alpha = -0.05 #Decay parameter of barrier
 
T = 10. #Time to expiry
T_0 = 3. #Time to vesting

lam_0 = 0.04 #Intensity of departure before vesting
lam = 0.04 #Intensity of departure after vesting

r = 0.05 #Risk-free rate
sig = 0.2 #Stock price volatility

#######################################################################################
import numpy as np
import math
from scipy.stats import multivariate_normal as mvn

r_a = r - alpha

b0 = np.sqrt((r + 0.5*(sig**2))**2 + 2*lam*(sig**2))
b_a = np.sqrt((r_a + 0.5*(sig**2))**2 + 2*lam*(sig**2))
    
c0 = np.sqrt((r - 0.5*(sig**2))**2 + 2*(lam + r)*(sig**2))
c_a = np.sqrt((r_a - 0.5*(sig**2))**2 + 2*(lam + r_a)*(sig**2))
cbar = np.sqrt((r_a - 0.5*(sig**2))**2 + 2*(lam + r)*(sig**2))
    
y_a = r_a - 0.5*(sig**2)

def Nd(x):
    
    return 0.5*(1 + math.erf(x/math.sqrt(2)))

def xY(Y):
    
    return np.log(Y/s)/(sig*np.sqrt(T_0)) - (r - 0.5*(sig**2))*(np.sqrt(T_0)/sig)

def B(a, b, c):
    
    mu_x = a
    mu_y = -a*b
    
    Mean = [mu_x, mu_y]
    
    sig_x = 1
    sig_y = 1 + b**2 
    p = -b/(np.sqrt(1 + b**2))
    
    Cov = [[sig_x, p*np.sqrt(sig_x*sig_y)], [p*np.sqrt(sig_x*sig_y), sig_y]]
    
    Mv = mvn(Mean, Cov)
    
    return np.exp((a**2)/2)*Mv.cdf([xY(L), c])


def BS(S, T, E, r, sigma):
    d1 = (math.log(S/E) + (r + 0.5*sigma**2)*T)/(sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    N1 = 0.5*(1 + math.erf(d1/math.sqrt(2)))
    N2 = 0.5*(1 + math.erf(d2/math.sqrt(2)))
    
    return S*N1 - E*math.exp(-r*T)*N2 

def C0(Y):
    
    return s*np.exp((r - 0.5*(sig**2))*T_0)*B(sig*np.sqrt(T_0), Q, d0(Y) + (np.sqrt(T - T_0)/sig)*(r_a + 0.5*(sig**2))) - K*np.exp(-alpha*(T - T_0))*np.exp(-r_a*(T - T_0))*B(0, Q, d0(Y) + (np.sqrt(T - T_0)/sig)*(r_a - 0.5*(sig**2)))

def C1(X):
    
    return L*((L/s)**(2*r_a/(sig**2)))*np.exp(((-2*r_a/(sig**2))*(r - 0.5*(sig**2))*T_0))*B((((-2*(r - alpha)))/sig)*np.sqrt(T_0), -Q, -d0((L**2)/X) + (np.sqrt(T - T_0)/sig)*(r - alpha + 0.5*(sig**2))) - K*np.exp(-alpha*(T - T_0))*((L/s)**(((2*(r - alpha)/(sig**2))) - 1))*np.exp(-1*(r - alpha)*(T - T_0) - ((2*(r - alpha)/(sig**2)) - 1)*(r - 0.5*(sig**2))*T_0)*B((((sig**2) - 2*(r - alpha))/sig)*np.sqrt(T_0), -Q, -d0((L**2)/X) + (np.sqrt(T - T_0)/sig)*(r - alpha - 0.5*(sig**2)))

def d0(Y):
    
    return (1/(sig*np.sqrt(T - T_0)))*(np.log(s/Y) + (r - 0.5*(sig**2))*T_0)

def d1(Y):
    
    return (1/(sig*np.sqrt(T - T_0)))*(np.log((L**2)/(s*Y)) - (r - 0.5*(sig**2))*T_0)

Q = np.sqrt(T_0/(T - T_0))

def D1(Y, R, r, b):
    
    A = -s*np.exp((R - 0.5*(sig**2))*T_0 - lam*(T - T_0))*B(sig*np.sqrt(T_0), Q, d0(Y) + (r + 0.5*(sig**2))*(np.sqrt(T - T_0)/sig)) 
    Bo = 0.5*(1 + (1/b)*(r + 0.5*(sig**2)))*s*((Y/s)**((r - b)/(sig**2) + 0.5))*np.exp(((b - r)/(sig**2) + 0.5)*(R - 0.5*(sig**2))*T_0)*B(((b - r)/sig + 0.5*sig)*np.sqrt(T_0), Q, d0(Y) + b*np.sqrt(T - T_0)/sig) 
    C = 0.5*(1 - (1/b)*(r + 0.5*(sig**2)))*s*((Y/s)**((r + b)/(sig**2) + 0.5))*np.exp((-(b + r)/(sig**2) + 0.5)*(R - 0.5*(sig**2))*T_0)*B((-(b + r)/sig + 0.5*sig)*np.sqrt(T_0), Q, d0(Y) - b*np.sqrt(T - T_0)/sig) 
    D = s*np.exp(R*T_0)*(Nd(xY(L) - sig*np.sqrt(T_0)) - Nd(xY(Y) - sig*np.sqrt(T_0))) 
    E = -0.5*(1 + (1/b)*(r + 0.5*(sig**2)))*s*((Y/s)**((r - b)/(sig**2) + 0.5))*np.exp(((b - r)/(sig**2) + 0.5)*(R - 0.5*(sig**2))*T_0 + (((b - r)/(sig**2) + 0.5)**2)*(0.5*(sig**2))*T_0)*(Nd(xY(L) - ((b - r)/sig + 0.5*sig)*np.sqrt(T_0)) - Nd(xY(Y) - ((b - r)/sig + 0.5*sig)*np.sqrt(T_0))) 
    F = -0.5*(1 - (1/b)*(r + 0.5*(sig**2)))*s*((Y/s)**((r + b)/(sig**2) + 0.5))*np.exp((-(b + r)/(sig**2) + 0.5)*(R - 0.5*(sig**2))*T_0 + ((-(b + r)/(sig**2) + 0.5)**2)*0.5*(sig**2)*T_0)*(Nd(xY(L) - (-(b + r)/sig + 0.5*sig)*np.sqrt(T_0)) - Nd(xY(Y) - (-(b + r)/sig + 0.5*sig)*np.sqrt(T_0)))
    
    return A + Bo + C + D + E + F

def D2(Y, R, r, c):
    
    A = np.exp(-1*(lam + R)*(T - T_0))*B(0, Q, d0(Y) + (r - 0.5*(sig**2))*(np.sqrt(T - T_0)/sig)) 
    Bo = - 0.5*(1 + (1/c)*(r - 0.5*(sig**2)))*((Y/s)**((r - c)/(sig**2) - 0.5))*np.exp(((c - r)/(sig**2) + 0.5)*(R - 0.5*(sig**2))*T_0)*B(((c - r)/sig + 0.5*sig)*np.sqrt(T_0), Q, d0(Y) + c*np.sqrt(T - T_0)/sig) 
    C = - 0.5*(1 - (1/c)*(r - 0.5*(sig**2)))*((Y/s)**((r + c)/(sig**2) - 0.5))*np.exp((-(c + r)/(sig**2) - 0.5)*(R - 0.5*(sig**2))*T_0)*B((-(c + r)/sig + 0.5*sig)*np.sqrt(T_0), Q, d0(Y) - c*np.sqrt(T - T_0)/sig) 
    D = - (Nd(xY(L)) - Nd(xY(Y))) 
    E = 0.5*(1 + (1/c)*(r - 0.5*(sig**2)))*((Y/s)**((r - c)/(sig**2) - 0.5))*np.exp(((c - r)/(sig**2) + 0.5)*(R - 0.5*(sig**2))*T_0 + (((c - r)/(sig**2) + 0.5)**2)*(0.5*(sig**2))*T_0)*(Nd(xY(L) - ((c - r)/sig + 0.5*sig)*np.sqrt(T_0)) - Nd(xY(Y) - ((c - r)/sig + 0.5*sig)*np.sqrt(T_0))) 
    F = 0.5*(1 - (1/c)*(r - 0.5*(sig**2)))*((Y/s)**((r + c)/(sig**2) - 0.5))*np.exp((-(c + r)/(sig**2) + 0.5)*(R - 0.5*(sig**2))*T_0 + ((-(c + r)/(sig**2) + 0.5)**2)*(0.5*(sig**2))*T_0)*(Nd(xY(L) - (-(c + r)/sig + 0.5*sig)*np.sqrt(T_0)) - Nd(xY(Y) - (-(c + r)/sig + 0.5*sig)*np.sqrt(T_0)))

    return ((lam*Y)/(lam + R))*(A + Bo + C + D + E + F)

def I1(x, T, z, r, b):
    
    ind1 = 0
    if x > z:
        ind1 = 1
    else:
        ind1 = 0
        
    ind2 = 0
    if x == z:
        ind2 = 1
    else:
        ind2 = 0
            
    D1 = (np.log(x/z))/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*(r + 0.5*(sig**2))
    D2 = (np.log(x/z))/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*b
    D3 = (np.log(x/z))/(sig*np.sqrt(T)) - (np.sqrt(T)/sig)*b
    
    A = (r + 0.5*(sig**2))/b
    B = x*((z/x)**((r-b)/(sig**2) + 0.5))
    C = (z/x)**(2*b/(sig**2))
           
    Int = x*(ind1 + 0.5*ind2 - np.exp(-lam*T)*Nd(D1)) + B*(A*(Nd(D2) - ind1 - 0.5*ind2) + 0.5*(1 - A)*(Nd(D2) + C*Nd(D3) - (1 + C)*ind1 - ind2) )
    return Int

def I2(x, T, z, R, r, c):
    
    ind1 = 0
    if x > z:
        ind1 = 1
    else:
        ind1 = 0
        
    ind2 = 0
    if x == z:
        ind2 = 1
    else:
        ind2 = 0
        
    D1 = (np.log(x/z))/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*(r - 0.5*(sig**2))
    D2 = (np.log(x/z))/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*c
    D3 = (np.log(x/z))/(sig*np.sqrt(T)) - (np.sqrt(T)/sig)*c
    
    A = (r - 0.5*(sig**2))/c
    B = ((z/x)**((r - c)/(sig**2) - 0.5))
    C = (z/x)**(2*c/(sig**2))
    
    LAM = lam*z/(lam + R)
    
    Int = -LAM*(ind1 + 0.5*ind2 - np.exp(-(lam + R)*T)*Nd(D1)) - LAM*B*(A*(Nd(D2) - ind1 - 0.5*ind2) + 0.5*(1 - A)*(Nd(D2) + C*Nd(D3) - (1 + C)*ind1 - ind2) )
    return Int

def G1(Y, R, r, b):
    
    A = -L*((L/s)**((2*r_a)/(sig**2)))*np.exp(((-2*r_a)/(sig**2))*(R - 0.5*(sig**2))*T_0 - lam*(T - T_0))*B((-2*r_a*np.sqrt(T_0))/sig, -Q, d1(Y) + (r + 0.5*(sig**2))*np.sqrt(T - T_0)/sig) 
    
    Bo = 0.5*(L**((2*(r_a + b - r))/(sig**2)))*(1 + (1/b)*(r + 0.5*(sig**2)))*(Y**((r - b)/(sig**2) + 0.5))*(s**((r - b - 2*r_a)/(sig**2) + 0.5))*np.exp((0.5 + (r - b - 2*r_a)/(sig**2))*(R - 0.5*(sig**2))*T_0)*B((0.5*sig + (r - b - 2*r_a)/sig)*np.sqrt(T_0), -Q, d1(Y) + b*np.sqrt(T - T_0)/sig)
    
    C = 0.5*(L**((2*(r_a - b - r))/(sig**2)))*(1 - (1/b)*(r + 0.5*(sig**2)))*(Y**((r + b)/(sig**2) + 0.5))*(s**((r + b - 2*r_a)/(sig**2) + 0.5))*np.exp((0.5 + (r + b - 2*r_a)/(sig**2))*(R - 0.5*(sig**2))*T_0)*B((0.5*sig + (r + b - 2*r_a)/sig)*np.sqrt(T_0), -Q, d1(Y) - b*np.sqrt(T - T_0)/sig)
    
    D = L*((L/s)**(2*r_a/(sig**2)))*np.exp(((-2*r_a)/(sig**2))*(R - 0.5*(sig**2))*T_0 + (2*(r_a**2)/(sig)**2)*T_0)*Nd(np.minimum(L, (L**2)/Y) + 2*r_a*np.sqrt(T_0)/sig)
    
    E = - 0.5*(L**((2*(r_a + b - r))/(sig**2)))*(1 + (1/b)*(r + 0.5*(sig**2)))*(Y**((r - b)/(sig**2) + 0.5))*(s**((r - b - 2*r_a)/(sig**2) + 0.5))*np.exp((0.5 + (r - b - 2*r_a)/(sig**2))*(R - 0.5*(sig**2))*T_0 + ((0.5 + (r - b - 2*r_a)/(sig**2))**2)*T_0*0.5*(sig**2))*Nd(np.minimum(L, (L**2)/Y) - np.sqrt(T_0)*(0.5*sig + (r - b - 2*r_a)/sig))
    
    F = - 0.5*(L**((2*(r_a - b - r))/(sig**2)))*(1 - (1/b)*(r + 0.5*(sig**2)))*(Y**((r + b)/(sig**2) + 0.5))*(s**((r + b - 2*r_a)/(sig**2) + 0.5))*np.exp((0.5 + (r + b - 2*r_a)/(sig**2))*(R - 0.5*(sig**2))*T_0 + ((0.5 + (r + b - 2*r_a)/(sig**2))**2)*T_0*0.5*(sig**2))*Nd(np.minimum(L, (L**2)/Y) - np.sqrt(T_0)*(0.5*sig + (r + b - 2*r_a)/sig)) 

    return A + Bo + C + D + E + F

def G2(Y, R, r, c):
    
    A = ((L/s)**((2*r_a)/(sig**2) - 1))*np.exp((1 - (2*r_a/(sig**2)))*(R - 0.5*(sig**2))*T_0 - (lam + R)*(T - T_0))*B((sig - 2*r_a/sig)*np.sqrt(T_0), -Q, d1(Y) + (r - 0.5*(sig**2))*np.sqrt(T - T_0)/sig) 

    Bo = - 0.5*(L**((2*(r_a + c - r))/(sig**2)))*(1 + (1/c)*(r - 0.5*(sig**2)))*(Y**((r - c)/(sig**2) - 0.5))*(s**((r - c - 2*r_a)/(sig**2) + 0.5))*np.exp((0.5 + (r - c - 2*r_a)/(sig**2))*(R - 0.5*(sig**2))*T_0)*B((0.5*sig + (r - c - 2*r_a)/sig)*np.sqrt(T_0), -Q, d1(Y) + c*np.sqrt(T - T_0)/sig) 
   
    C  = - 0.5*(L**((2*(r_a - c - r))/(sig**2)))*(1 - (1/c)*(r - 0.5*(sig**2)))*(Y**((r + c)/(sig**2) - 0.5))*(s**((r + c - 2*r_a)/(sig**2) + 0.5))*np.exp((0.5 + (r + c - 2*r_a)/(sig**2))*(R - 0.5*(sig**2))*T_0)*B((0.5*sig + (r + c - 2*r_a)/sig)*np.sqrt(T_0), -Q, d1(Y) - c*np.sqrt(T - T_0)/sig) 

    D = - ((L/s)**(2*r_a/(sig**2) - 1))*np.exp((1 - (2*r_a/(sig**2)))*(R - 0.5*(sig**2))*T_0 + ((1 - (2*r_a/(sig**2)))**2)*(sig**2)*0.5*T_0)*Nd(np.minimum(L, (L**2)/Y) - (sig - (2*r_a)/sig)*np.sqrt(T_0)) 
    
    E = 0.5*(L**((2*(r_a + c - r))/(sig**2)))*(1 + (1/c)*(r - 0.5*(sig**2)))*(Y**((r - c)/(sig**2) - 0.5))*(s**((r - c - 2*r_a)/(sig**2) + 0.5))*np.exp((0.5 + (r - c - 2*r_a)/(sig**2))*(R - 0.5*(sig**2))*T_0 + ((0.5 + (r - c - 2*r_a)/(sig**2))**2)*T_0*0.5*(sig**2))*Nd(np.minimum(L, (L**2)/Y) - np.sqrt(T_0)*(0.5*sig + (r - c - 2*r_a)/sig)) 
    
    F = 0.5*(L**((2*(r_a - c - r))/(sig**2)))*(1 - (1/c)*(r - 0.5*(sig**2)))*(Y**((r + c)/(sig**2) - 0.5))*(s**((r + c - 2*r_a)/(sig**2) + 0.5))*np.exp((0.5 + (r + c - 2*r_a)/(sig**2))*(R - 0.5*(sig**2))*T_0 + ((0.5 + (r + c - 2*r_a)/(sig**2))**2)*T_0*0.5*(sig**2))*Nd(np.minimum(L, (L**2)/Y) - np.sqrt(T_0)*(0.5*sig + (r + c - 2*r_a)/sig))

    return ((lam*Y)/(lam + R))*(A + Bo + C + D + E + F)

K_a = K*np.exp(-alpha*T)

Kbar_a = np.log(s/K_a)

K_ = np.log(s/K)
    
Lbar = np.log(s/L)

y1_a = r_a + (sig**2)/2
y2_a = r_a - (sig**2)/2

     
def P1(s):
    
    return s*Nd(Kbar_a/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*y1_a) - K_a*np.exp(-r_a*T)*Nd(Kbar_a/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*y2_a) - s*Nd(Lbar/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*y1_a) + K_a*np.exp(-r_a*T)*Nd(Lbar/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*y2_a) - ((L/s)**(2*r_a/(sig**2) - 1))*( ((L**2)/s)*Nd((1/(sig*np.sqrt(T)))*np.log((L**2)/(s*K_a)) + (np.sqrt(T)/sig)*y1_a) - K_a*np.exp(-r_a*T)*Nd((1/(sig*np.sqrt(T)))*np.log((L**2)/(s*K_a)) + (np.sqrt(T)/sig)*y2_a) - ((L**2)/s)*Nd(-Lbar/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*y1_a) + K_a*np.exp(-r_a*T)*Nd(-Lbar/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*y2_a))


def P(s, T, mu, y):
    
    return ( (L/s)**( (mu - y)/sig**2 ) )*Nd( (np.log(s/L))/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*y ) + ( (L/s)**( (mu + y)/sig**2 ) )*Nd( (np.log(s/L))/(sig*np.sqrt(T)) - (np.sqrt(T)/sig)*y )


def CaseB(s, T, K, r, sig, lam):

    X = I1(s, T, K, r, b0)

    Y = I2(s, T, K, r, r, c0)
    
    A = np.exp(-lam*T)*s*Nd(K_/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*(r + 0.5*(sig**2)))
    
    B = -K*np.exp(-(r + lam)*T)*Nd(K_/(sig*np.sqrt(T)) + (np.sqrt(T)/sig)*(r - 0.5*(sig**2)))
    
    return X + Y + A + B

def CaseC(s, K, L , alpha, T, lam, r, sig):
    
    A = np.exp(-lam*T)*P1(s) + L*P(s, T, y2_a, c_a) - K*P(s, T, y2_a, cbar)
    
    B = I1(s, T, K, r, b0) + I2(s, T, K, r, r, c0) - I1(s, T, L, r_a, b_a) - (K/L)*I2(s, T, L, r, r_a, cbar)
    
    C = ((L/s)**(2*r_a/(sig**2) - 1))*((I1((L**2)/s, T, K, r, b0)) + I2((L**2)/s, T, K, r, r, c0) - I1((L**2)/s, T, L, r_a, b_a) -(K/L)*I2((L**2)/s, T, L, r, r_a, cbar))
    
    return A + B - C

def CaseD(s, K, L , alpha, T, T_0, lam, lam_0, r, sig):
    
    K11 = L*np.exp(-alpha*T_0)*np.exp(-(r_a + lam_0)*T_0)*((L/s)**((y_a - c_a)/(sig**2)))*np.exp((r - 0.5*(sig**2))*T_0*((c_a - y_a)/(sig**2)))*B((c_a - y_a)*np.sqrt(T_0)/sig, Q, d0(L) + c_a*np.sqrt(T - T_0)/sig) - K*np.exp(-(r + lam_0)*T_0)*((L/s)**((y_a - cbar)/(sig**2)))*np.exp((r - 0.5*(sig**2))*T_0*((cbar - y_a)/(sig**2)))*B((cbar - y_a)*np.sqrt(T_0)/sig, Q, d0(L) + cbar*np.sqrt(T - T_0)/sig) + L*np.exp(-alpha*T_0)*np.exp(-(r_a + lam_0)*T_0)*((L/s)**((y_a + c_a)/(sig**2)))*np.exp((r - 0.5*(sig**2))*T_0*(-(c_a + y_a)/(sig**2)))*B(-(c_a + y_a)*np.sqrt(T_0)/sig, Q, d0(L) - c_a*np.sqrt(T - T_0)/sig) - K*np.exp(-(r + lam_0)*T_0)*((L/s)**((y_a + cbar)/(sig**2)))*np.exp((r - 0.5*(sig**2))*T_0*(-(cbar + y_a)/(sig**2)))*B(-(cbar + y_a)*np.sqrt(T_0)/sig, Q, d0(L) - cbar*np.sqrt(T - T_0)/sig)
    
    K12 = np.exp((lam - lam_0)*T_0)*np.exp(-lam*T_0)*(s*Nd(np.sqrt(T_0)*(r + 0.5*(sig**2))/sig + np.log(s/L)/(sig*np.sqrt(T_0))) - K*np.exp(-r*T_0)*Nd(np.sqrt(T_0)*(r - 0.5*(sig**2))/sig + np.log(s/L)/(sig*np.sqrt(T_0))))
    
    K2 = np.exp((lam - lam_0)*T_0)*np.exp(-(r + lam)*T_0)*(D1(K, r, r, b0) + D2(K, r, r, c0) - D1(L, r, r_a, b_a) - (K/L)*D2(L, r, r_a, cbar) - G1(K, r, r, b0) - G2(K, r, r, c0) + G1(L, r, r_a, b_a) + (K/L)*G2(L, r, r_a, cbar))
    
    K3 = np.exp((lam - lam_0)*T_0)*np.exp(-lam*T - r*T_0)*(C0(K*np.exp(-alpha*(T - T_0))) - C0(L) - C1(K*np.exp(-alpha*(T - T_0))) + C1(L))
    
    return K11 + K12 + K2 + K3

##########################################################

print(round(CaseB(s, T, K, r, sig, lam), 4))
print(round(CaseC(s, K, L , alpha, T, lam, r, sig), 4))
print(round(CaseD(s, K, L , alpha, T, T_0, lam, lam_0, r, sig), 4))

38.9753
20.2291
25.5889
